## Hybrid Recommender System

### İş Problemi
* ID'si verilen kullanıcı için item-based ve user-based recommender yöntemlerini kullanarak 10 film önerisi yapınız.

### Veri Seti Hikayesi
* Veri seti, bir film tavsiye hizmeti olan MovieLens tarafından sağlanmıştır. İçerisinde filmler ile birlikte bu filmlere yapılan derecelendirme puanlarını barındırmaktadır. 27.278 filmde 2.000.0263 derecelendirme içermektedir. Bu veri seti ise 17 Ekim 2016 tarihinde oluşturulmuştur. 138.493 kullanıcı ve 09 Ocak 1995 ile 31 Mart 2015 tarihleri arasında verileri içermektedir. Kullanıcılar rastgele seçilmiştir. Seçilen tüm kullanıcıların en az 20 filme oy verdiği bilgisi mevcuttur.

### Değişkenler
#### movie.csv veri seti için:
* movieid : Eşsiz film numarası
* title : Film adı
* genres : Tür

#### rating.csv veri seti için:
* userid : Eşsiz kullanıcı numarası (UniqueID)
* movieid : Eşsiz film numarası
* rating : Kullanıcı tarafından verilen puan
* timestamp : Değerlendirme tarihi

# 1) User-Based Recommendation

In [1]:
import pandas as pd

In [5]:
movie = pd.read_csv("movie.csv")
rating = pd.read_csv("rating.csv")
print(movie.head())
print(rating.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
(27278, 3)
   userId  movieId  rating            timestamp
0       1        2     3.5  2005-04-02 23:53:47
1       1       29     3.5  2005-04-02 23:31:16
2       1       32     3.5  2005-04-02 23:33:39
3       1       47     3.5  2005-04-02 23:32:07
4       1       50     3.5  2005-04-02 23:29:40
(20000263, 4)


* Bu iki veri setini ortak bir değişken kullanarak birbirlerine merge etmemiz gerekiyor. 
    * Bunun sebebi, analize daha rahat bir şekilde devam etmemizi sağlayacaktır.

In [9]:
df = movie.merge(rating, how = "left", on = "movieId")
df.head()
print(df.shape)
print(df["userId"].nunique()) # 138.493 tane benzersiz kullanıcı mevcut.
print(df["movieId"].nunique())# 27.278 tane benzersiz film id'si var.
print(df["title"].nunique()) # 27262 tane film adı bulunuyor.

(20000797, 6)
138493
27278
27262


In [11]:
df.isnull().sum()

movieId        0
title          0
genres         0
userId       534
rating       534
timestamp    534
dtype: int64

#### Adım 1) Veri setinde çok fazla film bulunuyor. Bu filmlerin bazılarının rate'leri çok düşük. Bu gibi filmleri kullanıcılara uyarmak istemeyiz. Bu yüzden bir sınır belirlememiz gerekebilir. Ben, daha rahat çalışabilmek adına bu seviyeyi biraz yüksek tutacağım.

In [13]:
# Her bir filmin kaçar kere oylandığı bizim için önemli. Bu yüzden bir değişkene atayalım.
comment_counts = pd.DataFrame(df["title"].value_counts())
print(comment_counts)

                                           count
title                                           
Pulp Fiction (1994)                        67310
Forrest Gump (1994)                        66172
Shawshank Redemption, The (1994)           63366
Silence of the Lambs, The (1991)           63299
Jurassic Park (1993)                       59715
...                                          ...
Rapture (Arrebato) (1980)                      1
Education of Mohammad Hussein, The (2013)      1
Satanas (2007)                                 1
Psychosis (2010)                               1
Innocence (2014)                               1

[27262 rows x 1 columns]


In [20]:
# Toplam oy kullanılma sayısı 10.000'in altında olan filmleri ayıracağız.
print("Sınır belirlenmeden önceki satır sayısı : ",df.shape)
rare_movies = comment_counts[comment_counts["count"] <= 10000].index
common_movies = df[~df["title"].isin(rare_movies)]
print("Sınır belirlendikten sonraki satır sayısı: ",common_movies.shape)

# Veri setinde bulunan satırların çoğunu elemiş olduk.

Sınır belirlenmeden önceki satır sayısı :  (20000797, 6)
Sınır belirlendikten sonraki satır sayısı:  (9050403, 6)


#### Adım 2) Bir pivot table oluşturacağız. Bu pivot table'da :
* Sütun    : Film isimlerri 
* Satır    : Kullanıcılar
* Değerler : Rating Puanları

In [21]:
user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
user_movie_df.head()

title,10 Things I Hate About You (1999),12 Angry Men (1957),2001: A Space Odyssey (1968),28 Days Later (2002),300 (2007),A.I. Artificial Intelligence (2001),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Addams Family Values (1993),...,Wild Wild West (1999),William Shakespeare's Romeo + Juliet (1996),Willy Wonka & the Chocolate Factory (1971),Witness (1985),"Wizard of Oz, The (1939)","X-Files: Fight the Future, The (1998)",X-Men (2000),X2: X-Men United (2003),You've Got Mail (1998),Young Frankenstein (1974)
userId,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,3.5,3.5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.5,NaN,NaN,4.0,NaN,4.0
2.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,5.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,5.0,4.0,4.0,5.0,NaN,NaN,NaN,5.0
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* Her bir kullanıcının izlediği ve izlemediği filmleri buradan görebiliyoruz. Bazı değerler mevcut. Bu değerler, kullanıcıların filmlere verdiği rate puanları.

#### Adım 3) Bu veri seti içerisinden öneri yapacağımız rastgele bir kullanıcı seçmemiz gerekiyor.  Sonrasında bu kullanıcının bulunduğu satırı belirleyeceğiz. Kullanıcının izlediği filmleri belirlemek, bu aşamadakı en önemli aşamadır.

In [33]:
random_user = 108170
random_user_df = user_movie_df[user_movie_df.index == random_user]
print(random_user_df.shape)
random_user_df.style.background_gradient(cmap="YlGnBu")

(1, 462)


/Users/hasangulec/anaconda3/lib/python3.11/site-packages/pandas/io/formats/style.py:3819: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(gmap) if vmin is None else vmin
/Users/hasangulec/anaconda3/lib/python3.11/site-packages/pandas/io/formats/style.py:3820: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(gmap) if vmax is None else vmax


title,10 Things I Hate About You (1999),12 Angry Men (1957),2001: A Space Odyssey (1968),28 Days Later (2002),300 (2007),A.I. Artificial Intelligence (2001),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Addams Family Values (1993),"Adventures of Priscilla, Queen of the Desert, The (1994)","African Queen, The (1951)",Air Force One (1997),Airplane! (1980),Aladdin (1992),Alien (1979),Alien: Resurrection (1997),Aliens (1986),Alien³ (a.k.a. Alien 3) (1992),Almost Famous (2000),Amadeus (1984),"Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)",American Beauty (1999),American History X (1998),American Pie (1999),"American President, The (1995)",Animal House (1978),Annie Hall (1977),Antz (1998),Apocalypse Now (1979),Apollo 13 (1995),Arachnophobia (1990),Armageddon (1998),Army of Darkness (1993),As Good as It Gets (1997),Austin Powers: International Man of Mystery (1997),Austin Powers: The Spy Who Shagged Me (1999),Babe (1995),Back to the Future (1985),Back to the Future Part II (1989),Back to the Future Part III (1990),Bad Boys (1995),Basic Instinct (1992),Batman (1989),Batman Begins (2005),Batman Forever (1995),Batman Returns (1992),"Beautiful Mind, A (2001)",Beauty and the Beast (1991),Beetlejuice (1988),Being John Malkovich (1999),Best in Show (2000),Beverly Hills Cop (1984),Beverly Hills Cop III (1994),Big (1988),Big Fish (2003),"Big Lebowski, The (1998)","Birdcage, The (1996)","Birds, The (1963)",Black Hawk Down (2001),Blade (1998),Blade Runner (1982),"Blair Witch Project, The (1999)",Blazing Saddles (1974),"Blues Brothers, The (1980)",Boogie Nights (1997),"Boot, Das (Boat, The) (1981)","Bourne Identity, The (2002)","Bourne Supremacy, The (2004)","Bourne Ultimatum, The (2007)",Bowling for Columbine (2002),Braveheart (1995),Brazil (1985),"Breakfast Club, The (1985)","Bridge on the River Kwai, The (1957)",Bridget Jones's Diary (2001),Broken Arrow (1996),"Bug's Life, A (1998)",Butch Cassidy and the Sundance Kid (1969),"Cable Guy, The (1996)",Casablanca (1942),Casino (1995),Casino Royale (2006),Casper (1995),Cast Away (2000),Catch Me If You Can (2002),Charlie's Angels (2000),Chasing Amy (1997),Chicken Run (2000),Chinatown (1974),Chocolat (2000),"Christmas Story, A (1983)",Citizen Kane (1941),City Slickers II: The Legend of Curly's Gold (1994),City of God (Cidade de Deus) (2002),Clear and Present Danger (1994),Clerks (1994),"Client, The (1994)",Cliffhanger (1993),"Clockwork Orange, A (1971)",Close Encounters of the Third Kind (1977),Clueless (1995),Cocoon (1985),Con Air (1997),Coneheads (1993),Congo (1995),Conspiracy Theory (1997),Contact (1997),Cool Hand Luke (1967),Crimson Tide (1995),Crocodile Dundee (1986),"Crouching Tiger, Hidden Dragon (Wo hu cang long) (2000)","Crow, The (1994)","Crying Game, The (1992)",Dances with Wolves (1990),Dark City (1998),"Dark Knight, The (2008)",Dave (1993),Dead Man Walking (1995),Dead Poets Society (1989),Deep Impact (1998),"Deer Hunter, The (1978)",Demolition Man (1993),"Departed, The (2006)",Desperado (1995),"Devil's Advocate, The (1997)",Die Hard (1988),Die Hard 2 (1990),Die Hard: With a Vengeance (1995),Dirty Dancing (1987),Disclosure (1994),Dogma (1999),Donnie Brasco (1997),Donnie Darko (2001),Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),Dracula (Bram Stoker's Dracula) (1992),Dragonheart (1996),Dumb & Dumber (Dumb and Dumber) (1994),E.T. the Extra-Terrestrial (1982),Ed Wood (1994),Edward Scissorhands (1990),Election (1999),Enemy of the State (1998),"English Patient, The (1996)",Eraser (1996),Erin Brockovich (2000),Eternal Sunshine of the Spotless Mind (2004),Executive Decision (1996),"Exorcist, The (1973)",Eyes Wide Shut (1999),Face/Off (1997),Fantasia (1940),Fargo (1996),Father of the Bride Part II (1995),Ferris Bueller's Day Off (1986),"Few Good Men, A (1992)",Field of Dreams (1989),"Fifth Element, The (1997)",Fight Club (1999),Finding Nemo (2003),"Firm, The (1993)",First Knight (1995),"Fish Called Wanda, A (1988)"

In [36]:
movies_watched = random_user_df.columns[random_user_df.notna().any()].to_list()
print(movies_watched)
print(movie.columns[movie.notna().any()].to_list())

['2001: A Space Odyssey (1968)', 'Adventures of Priscilla, Queen of the Desert, The (1994)', 'Aladdin (1992)', 'Aliens (1986)', 'Almost Famous (2000)', 'Amadeus (1984)', 'American History X (1998)', 'Arachnophobia (1990)', 'Army of Darkness (1993)', 'Basic Instinct (1992)', 'Batman Returns (1992)', 'Big Lebowski, The (1998)', 'Blair Witch Project, The (1999)', 'Blues Brothers, The (1980)', 'Bourne Identity, The (2002)', 'Braveheart (1995)', 'Breakfast Club, The (1985)', "Charlie's Angels (2000)", "City Slickers II: The Legend of Curly's Gold (1994)", 'Clear and Present Danger (1994)', 'Clerks (1994)', 'Clockwork Orange, A (1971)', 'Con Air (1997)', 'Contact (1997)', 'Crocodile Dundee (1986)', 'Crying Game, The (1992)', 'Dead Poets Society (1989)', 'Demolition Man (1993)', 'Desperado (1995)', "Devil's Advocate, The (1997)", 'Donnie Darko (2001)', 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)', 'Edward Scissorhands (1990)', 'Eternal Sunshine of the Spotless

#### Adım 4) Aynı filmleri izleyen diğer kullanıcıların Verisine ve ID'sine erişelim ve farklı bir df'e atayalım.

In [38]:
movies_watched_df = user_movie_df[movies_watched]
print(movies_watched_df.head())
print(movies_watched_df.shape)

title   2001: A Space Odyssey (1968)  \
userId                                 
1.0                              3.5   
2.0                              5.0   
3.0                              5.0   
4.0                              NaN   
5.0                              NaN   

title   Adventures of Priscilla, Queen of the Desert, The (1994)  \
userId                                                             
1.0                                                   NaN          
2.0                                                   NaN          
3.0                                                   NaN          
4.0                                                   NaN          
5.0                                                   NaN          

title   Aladdin (1992)  Aliens (1986)  Almost Famous (2000)  Amadeus (1984)  \
userId                                                                        
1.0                NaN            4.0                   NaN             NaN   
2.0  

In [40]:
# Her bir user, seçili filmlerden kaçını izledi bunun bilgisine ulaşmak istersek
user_movie_count = movies_watched_df.T.notnull().sum()
user_movie_count = user_movie_count.reset_index()
user_movie_count.columns = ["userId", "movie_count"]
user_movie_count.head(5)
user_movie_count.shape

(137658, 2)

In [59]:
# Hala çok fazla kullanıcı bulunuyor. Bu yüzden, bir filtreleme daha koyuyoruz. 
# Seçilen kullanıcının oy verdiği filmlerin yüzde 80 ve üstünü izleyenleri benzer kullanıcılar olarak görelim.

perc = len(movies_watched) * 60 / 100
users_same_movies = user_movie_count[user_movie_count["movie_count"] > perc]["userId"]
len(users_same_movies)

4902

#### Adım 5) Öneri yapılacak kullanıcı ile en benzer kullanıcıları seçelim.

In [60]:
final_df = movies_watched_df[movies_watched_df.index.isin(users_same_movies)]
print(final_df.head())
print(final_df.shape)

title   2001: A Space Odyssey (1968)  \
userId                                 
58.0                             5.0   
91.0                             2.5   
116.0                            NaN   
147.0                            3.5   
156.0                            NaN   

title   Adventures of Priscilla, Queen of the Desert, The (1994)  \
userId                                                             
58.0                                                  NaN          
91.0                                                  4.0          
116.0                                                 NaN          
147.0                                                 NaN          
156.0                                                 3.0          

title   Aladdin (1992)  Aliens (1986)  Almost Famous (2000)  Amadeus (1984)  \
userId                                                                        
58.0               NaN            4.0                   NaN             4.0   
91.0 

In [61]:
# Kullanıcıların birbirleri ile olan korelasyonlarının bulunacağı bir korelasyon df'i oluşturalım.
# Korelasyon kullanma sebebimiz, korelasyon katsayılarını kullanarak bir film önerisinde bulunacağız.
corr_df = final_df.T.corr().unstack().sort_values()
corr_df = pd.DataFrame(corr_df, columns=["corr"])
corr_df.index.names = ['user_id_1', 'user_id_2']
corr_df = corr_df.reset_index()
corr_df

,user_id_1,user_id_2,corr
0,21853.0,79828.0,-0.592080
1,79828.0,21853.0,-0.592080
2,72838.0,110130.0,-0.580457
3,110130.0,72838.0,-0.580457
4,56879.0,62340.0,-0.576827
...,...,...,...
24029599,132586.0,1376.0,NaN
24029600,134290.0,1376.0,NaN
24029601,135857.0,1376.0,NaN
24029602,137746.0,1376.0,NaN


In [66]:
# Korelasyon değerli 0.65 üzerinde olan kullanıcıları bırakacak şekilde bir filtreleme daha yapalım.
top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] >= 0.65)][["user_id_2", "corr"]].reset_index(drop=True)
top_users = top_users.sort_values(by='corr', ascending=False)
top_users.rename(columns={"user_id_2": "userId"}, inplace=True)
top_users.shape
top_users

,userId,corr
24,108170.0,1.000000
23,89195.0,0.737658
22,11517.0,0.719326
21,5155.0,0.710507
20,121747.0,0.704917
19,82860.0,0.702602
18,44435.0,0.694021
17,23753.0,0.685583
16,89202.0,0.678307
15,42497.0,0.677204


In [67]:
# Korelasyon değerlerinin yanına rating değerlerinide ekleyelim. 
# Bunun sebebi, korelasyon ve Rating değerlerini bir formülde kullanacak olmamız.

top_users_ratings = top_users.merge(rating[["userId", "movieId", "rating"]], how='inner')
top_users_ratings = top_users_ratings[top_users_ratings["userId"] != random_user]
top_users_ratings["userId"].unique()
top_users_ratings.head()

,userId,corr,movieId,rating
194,89195.0,0.737658,1,2.0
195,89195.0,0.737658,2,3.0
196,89195.0,0.737658,6,5.0
197,89195.0,0.737658,10,5.0
198,89195.0,0.737658,16,4.5


#### Adım 6) Weighted Average Recommendation Score'un Hesaplanması ve İlk 5 Filmin Tutulması
* Weighted Average Recommendation Score hesaplamamız için, 
    * corr * rating formülünü kullanmamız gerekiyor.

In [68]:
top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']
top_users_ratings.head()

,userId,corr,movieId,rating,weighted_rating
194,89195.0,0.737658,1,2.0,1.475316
195,89195.0,0.737658,2,3.0,2.212974
196,89195.0,0.737658,6,5.0,3.688290
197,89195.0,0.737658,10,5.0,3.688290
198,89195.0,0.737658,16,4.5,3.319461


In [69]:
# Kullanıcı için bir öneri yapmak istiyorsak, Weighted Rating Score'ların ortalamalarını almamız gerek
recommendation_df = top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})
recommendation_df = recommendation_df.reset_index()
recommendation_df.head()

,movieId,weighted_rating
0,1,1.980258
1,2,1.275607
2,3,1.879999
3,5,1.729232
4,6,2.844900


In [75]:
# Ortalama weighted_rating puanlarından 3.5 puan üzeri olanları filtreleyelim.
recommendation_df[recommendation_df["weighted_rating"] > 3.5]
movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 3.5].sort_values("weighted_rating", ascending=False)
print(movies_to_be_recommend)

      movieId  weighted_rating
2855    43832         3.688290
3104    57910         3.688290
2089     5909         3.688290
3241    71057         3.688290
2586    26326         3.688290
2606    26736         3.688290
3229    69140         3.688290
2674    30745         3.688290
1801     4713         3.688290
3186    64620         3.688290
3004    52831         3.688290
330       671         3.620412
2988    51559         3.596629
2607    26776         3.596629
2412     7502         3.596629
3386    93838         3.596629
3458   105844         3.596629


In [76]:
# Film isimlerini görmek istiyorsak eğer,
print(movies_to_be_recommend.merge(movie[["movieId", "title"]])["title"][:5])

0                        Call of Cthulhu, The (2005)
1                                       Teeth (2007)
2                        Visitor Q (Bizita Q) (2001)
3                                           9 (2009)
4    Holy Mountain, The (Montaña sagrada, La) (1973)
Name: title, dtype: object


* Kısaca şunu söylemeliyiz ki,
    * Kullanıcı bazlı bir tavsiye öneri sistemi kurduğumuzda, seçilen rastgele kullanıcıya izlemediği filmlerden önerebileceğimiz ilk 5 filmi yukarıdaki çıktıda görebilirsiniz.

# 2) Item-Based Recommendation

#### Adım 1) Kullanıcının en son izlediği ve en yüksek puan verdiği filmin adına göre item-based önerisi yapalım.

In [87]:
# Öneri yapılacak kullanıcının 5 puan verdiği filmlerden puanı en güncel olan filmid'sini alalım.
user = 105171
movie_id = rating[(rating["userId"] == user) & (rating["rating"] == 5.0)].sort_values(by="timestamp", ascending=False)["movieId"][0:1].values[0]
print(movie_id)

2959


In [89]:
# Seçilen movieId'ye göre filtreleme yapalım.
movie[movie["movieId"] == movie_id]["title"].values[0]
movie_df = user_movie_df[movie[movie["movieId"] == movie_id]["title"].values[0]]
movie_df

userId
1.0         4.0
2.0         NaN
3.0         NaN
4.0         NaN
5.0         NaN
           ... 
138489.0    4.5
138490.0    NaN
138491.0    NaN
138492.0    NaN
138493.0    5.0
Name: Fight Club (1999), Length: 137658, dtype: float64

#### Adım 2) Filtrelenen dataframe’i kullanarak seçili filmle diğer filmlerin korelasyonunu bulunuz ve sıralayınız.

In [90]:
user_movie_df.corrwith(movie_df).sort_values(ascending=False).head(10)

title
Fight Club (1999)                            1.000000
Seven (a.k.a. Se7en) (1995)                  0.349805
Pulp Fiction (1994)                          0.328060
Twelve Monkeys (a.k.a. 12 Monkeys) (1995)    0.327986
Snatch (2000)                                0.326288
American History X (1998)                    0.326111
Trainspotting (1996)                         0.322369
Lock, Stock & Two Smoking Barrels (1998)     0.319509
Memento (2000)                               0.308780
Donnie Darko (2001)                          0.305932
dtype: float64

#### Adım 3) Seçili film’in kendisi haricinde ilk 5 film’I öneri olarak veriniz.

In [95]:
# Bir fonksiyon haline getirelim.
def item_based_recommender(movie_name, user_movie_df):
    movie = user_movie_df[movie_name]
    return user_movie_df.corrwith(movie).sort_values(ascending=False).head(10)

In [96]:
movies_from_item_based = item_based_recommender(movie[movie["movieId"] == movie_id]["title"].values[0], user_movie_df)
movies_from_item_based[1:6].index

Index(['Seven (a.k.a. Se7en) (1995)', 'Pulp Fiction (1994)',
       'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)', 'Snatch (2000)',
       'American History X (1998)'],
      dtype='object', name='title')